<a href="https://colab.research.google.com/github/ckirby04/UArk-Projects/blob/WDA/WDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

# Custom Dataset
class WeldDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        img_path = os.path.join(self.folder_path, filename)
        image = Image.open(img_path).convert('RGB')
        depth = float(os.path.splitext(filename)[0])

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor([depth], dtype=torch.float32)

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and splits
dataset = WeldDataset("path_to_your_folder", transform=transform)
total_len = len(dataset)
train_len = int(0.7 * total_len)
val_len = int(0.1 * total_len)
test_len = total_len - train_len - val_len
train_set, val_set, test_set = random_split(dataset, [train_len, val_len, test_len])
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16)
test_loader = DataLoader(test_set, batch_size=16)

# Model
class ResNetRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 1)

    def forward(self, x):
        return self.backbone(x)

model = ResNetRegression()
criterion = nn.MSELoss()
mae_metric = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  # You can increase this
    model.train()
    total_mse, total_mae = 0, 0
    for imgs, targets in train_loader:
        imgs, targets = imgs.to(device), targets.to(device)
        outputs = model(imgs)
        mse = criterion(outputs, targets)
        mae = mae_metric(outputs, targets)
        optimizer.zero_grad()
        mse.backward()
        optimizer.step()
        total_mse += mse.item()
        total_mae += mae.item()
    avg_mse = total_mse / len(train_loader)
    avg_mae = total_mae / len(train_loader)

    # Validation
    model.eval()
    with torch.no_grad():
        val_mse, val_mae = 0, 0
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            val_mse += criterion(outputs, targets).item()
            val_mae += mae_metric(outputs, targets).item()
        val_mse /= len(val_loader)
        val_mae /= len(val_loader)

    print(f"Epoch {epoch+1}: Train MSE={avg_mse:.4f}, MAE={avg_mae:.4f} | Val MSE={val_mse:.4f}, MAE={val_mae:.4f}")

# Final Evaluation on Test Set
model.eval()
with torch.no_grad():
    test_mse, test_mae = 0, 0
    for imgs, targets in test_loader:
        imgs, targets = imgs.to(device), targets.to(device)
        outputs = model(imgs)
        test_mse += criterion(outputs, targets).item()
        test_mae += mae_metric(outputs, targets).item()
    test_mse /= len(test_loader)
    test_mae /= len(test_loader)

print(f"Test MSE: {test_mse:.4f}, Test MAE: {test_mae:.4f}")
